In [ ]:
import re                         #正規表現reモジュール（文字列の中から特定のパターンを探す）
import math                       #数学関数mathモジュール
import time                       #時間の処理を扱うtimeモジュール
from urllib.parse import urljoin  #urlを分解・結合・変換を行うモジュール

import requests                   #HTTPのリクエスト用ライブラリ（webページへアクセスして、HTML等を取得）
from bs4 import BeautifulSoup     #webページのHTMLを解析・抽出するライブラリ
import pandas as pd               #表・テーブル・行、列のようなデータを扱ったり、整理するライブラリ
import psycopg2                   #PythonとPostgreSQLを接続するドライバ（接続ライブラリ）


In [2]:
#PostgreSQLの接続情報
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASSWORD = 'ja52712'
DB_HOST = 'localhost'
DB_PORT = '5432'

In [ ]:
#ウェブスクレイピングの練習サイトを利用
BASE = 'https://books.toscrape.com/'
HEADERS = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Scraping-Practice/1.0'
}

In [ ]:
#型ヒント（:str）を使用した関数parse_priceを定義
def parse_price(text:str):
    if not isinstance(text,str):#isinstance関数を使ってオブジェクトが指定した型かどうかを判定
        return None,None #戻り値がタプルなのでNoneを2つ設定
    symbol_map = {"£": "GBP", "€": "EUR", "$": "USD"}#通貨記号を判定するという想定で対応表を用意
    currency = None#初期値(None)を代入
    for sym,cur in symbol_map.items():#辞書を記号、通貨コードのペアで順に取り出しループ処理
        if sym in text:#textにその記号が含まれているかをチェック
            currency = cur#含まれていたらcurrencyに対応コードをセット
            break#セットされたらループ処理を終了
    m = re.search(r"\d+(?:\.\d+)?", text.replace(",", ""))#価格文字列からカンマ（桁区切り）を削除してから、正規表現で 最初に現れる数値（整数 or 小数） を1件だけ検索。
    return (float(m.group(0)),currency) if m else (None,currency)#数値が見つかったら文字列を float に変換して (価格, 通貨コード) を返す。見つからなければ価格は None のまま (None, 通貨コード) を返す（通貨だけ分かるケースに対応）。

In [ ]:
def scrape_list_page(url:str):#型ヒントを使った関数定義
    r = requests.get(url,headers=HEADERS,timeout=15)#HTTPリクエストを送って、レスポンスをrに格納
    r.raise_for_status()#HTTPエラーの時に自動でHTTPErrorを投げる。エラーチェック。
    soup = BeautifulSoup(r.text,'html.parser')#HTML文字列を解析オブジェクトに変換する。
    
    rows = []
    for li in soup.select('section div ol.row li'):
        a = li.select_one('h3 a')
        title = a.get('title')
        href = a.get('href')
        product_url = urljoin(url,href)
        
        price_text = li.select_one('.price_color').get_text(strip=True)
        price,currency = parse_price(price_text)
        
        stock_text = li.select_one('.availability').get_text(strip=True)
        
        raiting = None
        raiting_el = li.select_one('.star-rating')
        
        if raiting_el and raiting_el.has_attr('class'):
            for c in raiting_el['class']:
                if c in {"One","Two","Three","Four","Five"}:
                    raiting = c
                    
                    
        rows.append({
            "site": "books.toscrape",
            "title": title,
            "price": price,
            "currency": currency,
            "availability": stock_text,
            "rating": raiting,
            "url": product_url,
        })
        
    next_href = soup.select_one('li.next a')
    next_url = urljoin(url,next_href.get('href')) if next_href else None
    return rows,next_url

In [6]:
all_rows = []
url = BASE
pages = 0
MAX_PAGES = 5

while url and pages < MAX_PAGES:
    rows,url = scrape_list_page(url)
    all_rows.extend(rows)
    pages += 1
    time.sleep(0.8)
    
    
df = pd.DataFrame(all_rows)
display(df.head(10))
print('件数:',len(df))
print('欠損数:\n',df.isna().sum())

,site,title,price,currency,availability,rating,url
0,books.toscrape,A Light in the Attic,51.77,GBP,In stock,Three,https://books.toscrape.com/catalogue/a-light-i...
1,books.toscrape,In Her Wake,12.84,GBP,In stock,One,https://books.toscrape.com/catalogue/in-her-wa...
2,books.toscrape,Slow States of Collapse: Poems,57.31,GBP,In stock,Three,https://books.toscrape.com/catalogue/slow-stat...
3,books.toscrape,The Nameless City (The Nameless City #1),38.16,GBP,In stock,Four,https://books.toscrape.com/catalogue/the-namel...
4,books.toscrape,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,GBP,In stock,Five,https://books.toscrape.com/catalogue/princess-...


件数: 5
欠損数:
 site            0
title           0
price           0
currency        0
availability    0
rating          0
url             0
dtype: int64


In [7]:
create_sql = """
CREATE TABLE IF NOT EXISTS public.ec_prices (
    id          SERIAL PRIMARY KEY,
    site        TEXT NOT NULL,
    title       TEXT,
    url         TEXT UNIQUE,
    price       NUMERIC(12,2),
    currency    TEXT,
    availability TEXT,
    rating      TEXT,
    scraped_at  TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);
"""

In [8]:
upsert_sql = """
INSERT INTO public.ec_prices (site, title, url, price, currency, availability, rating)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (url)
DO UPDATE SET
    site        = EXCLUDED.site,
    title       = EXCLUDED.title,
    price       = EXCLUDED.price,
    currency    = EXCLUDED.currency,
    availability= EXCLUDED.availability,
    rating      = EXCLUDED.rating,
    scraped_at  = CURRENT_TIMESTAMP;
"""

In [9]:
rows = (
    df[["site","title","url","price","currency","availability","rating"]]
    .where(pd.notnull(df), None)
    .values.tolist()
)

with psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD,
    host=DB_HOST, port=DB_PORT
) as conn:
    with conn.cursor() as cur:
        cur.execute(create_sql)
        cur.executemany(upsert_sql, rows)
    conn.commit()

print(f"保存（挿入/更新）完了：{len(rows)} 件")

保存（挿入/更新）完了：5 件


In [10]:
with psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD,
    host=DB_HOST, port=DB_PORT
) as conn:
    preview = pd.read_sql(
        """
        SELECT id, site, title, price, currency, rating, availability, url, scraped_at
        FROM public.ec_prices
        ORDER BY scraped_at DESC, id DESC
        LIMIT 15
        """,
        conn
    )

display(preview)


C:\Users\ja060\AppData\Local\Temp\ipykernel_31088\3447809237.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preview = pd.read_sql(


,id,site,title,price,currency,rating,availability,url,scraped_at
0,5,books.toscrape,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,GBP,Five,In stock,https://books.toscrape.com/catalogue/princess-...,2025-10-09 18:25:11.048413
1,4,books.toscrape,The Nameless City (The Nameless City #1),38.16,GBP,Four,In stock,https://books.toscrape.com/catalogue/the-namel...,2025-10-09 18:25:11.048413
2,3,books.toscrape,Slow States of Collapse: Poems,57.31,GBP,Three,In stock,https://books.toscrape.com/catalogue/slow-stat...,2025-10-09 18:25:11.048413
3,2,books.toscrape,In Her Wake,12.84,GBP,One,In stock,https://books.toscrape.com/catalogue/in-her-wa...,2025-10-09 18:25:11.048413
4,1,books.toscrape,A Light in the Attic,51.77,GBP,Three,In stock,https://books.toscrape.com/catalogue/a-light-i...,2025-10-09 18:25:11.048413
